In [2]:
import os
import gc
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import scipy.stats  as ss

from statsmodels.api import ProbPlot


warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_rows', None)

In [3]:
train_df = pd.read_csv("../dataset/Cleaned_train.csv")

In [4]:
# 类别、离散特征
cate_col_list = ['ProductCD','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','addr1','addr2']

#循环加入特征
for i in range(1,7):
    cate_col_list.append('card' + str(i))
for i in range(1 ,10):
    cate_col_list.append('M' + str(i))
for i in range(12 ,39):
    cate_col_list.append('id_' + str(i // 10) + str(i % 10))
#连续型特征
num_col_list = [x for x in train_df.columns if x not in cate_col_list]
num_col_list.remove('isFraud')

In [25]:
for col in cate_col_list:
    if train_df[col].dtype == 'object':
        print(col)

ProductCD
P_emaildomain
R_emaildomain
DeviceType
DeviceInfo
card4
card6
M1
M2
M3
M4
M5
M6
M7
M8
M9
id_12
id_16
id_23
id_27
id_28
id_29
id_30
id_31
id_33
id_34
id_35
id_36
id_37
id_38


In [28]:
train_df['card4'].value_counts()

visa                384767
mastercard          189217
american express      8328
discover              6651
other                 1577
Name: card4, dtype: int64

# Feature Engineering

- 目的并不是要做这个项目到极致，因为采用的是集成学习这种复杂模型，暂时不必采用分箱处理
- 采用树模型，暂时也不必做数据缩放处理

## Feature Encoding

In [7]:
train_df['isFraud'].groupby(['id_01']).agg('mean')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\envs\kaggle\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Mr.Li\AppData\Local\Temp\ipykernel_13184\1604211498.py", line 1, in <module>
    train_df['isFraud'].groupby(['id_01']).agg('mean')
  File "D:\anaconda\envs\kaggle\lib\site-packages\pandas\core\series.py", line 1894, in groupby
    dropna=dropna,
  File "D:\anaconda\envs\kaggle\lib\site-packages\pandas\core\groupby\groupby.py", line 897, in __init__
    dropna=self.dropna,
  File "D:\anaconda\envs\kaggle\lib\site-packages\pandas\core\groupby\grouper.py", line 862, in get_grouper
    raise KeyError(gpr)
KeyError: 'id_01'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\envs\kaggle\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
Attri

TypeError: object of type 'NoneType' has no len()

In [10]:
temp_df = train_df.iloc[:1000,:].copy()

In [29]:
temp_df.groupby(['card4'])['isFraud'].mean()

card4
american express    0.000000
discover            0.000000
mastercard          0.023952
visa                0.010802
Name: isFraud, dtype: float64

In [30]:
temp_df['card4'].groupby(temp_df['id_01']).mean()


TypeError: Could not convert visavisavisavisamastercardmastercard to numeric

In [22]:
from feature_engine.encoding import MeanEncoder

In [31]:
me = MeanEncoder(['card4'])

In [33]:
temp_df['card4'].value_counts()

visa                648
mastercard          334
american express     10
discover              8
Name: card4, dtype: int64

In [32]:
me.fit_transform(temp_df,temp_df['isFraud'])

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000.0,0,86400.0,68.500,W,9999,834.379332,150.0,0.000000,9999.000000,...,other,35.212434,other,other,other,other,other,other,other,other
1,2987001.0,0,86401.0,29.000,W,9999,404.000000,150.0,0.023952,-9999.000000,...,other,35.212434,other,other,other,other,other,other,other,other
2,2987002.0,0,86469.0,59.000,W,4663,490.000000,150.0,0.010802,166.000000,...,other,35.212434,other,other,other,other,other,other,other,other
3,2987003.0,0,86499.0,50.000,W,18132,567.000000,150.0,0.023952,-9999.000000,...,other,35.212434,other,other,other,other,other,other,other,other
4,2987004.0,0,86506.0,50.000,H,9999,514.000000,150.0,0.023952,-9999.000000,...,samsung browser 6.2,32.000000,other,match_status:2,T,F,T,T,mobile,other
5,2987005.0,0,86510.0,49.000,W,9999,555.000000,150.0,0.010802,226.000000,...,other,35.212434,other,other,other,other,other,other,other,other
6,2987006.0,0,86522.0,159.000,W,9999,360.000000,150.0,0.010802,166.000000,...,other,35.212434,other,other,other,other,other,other,other,other
7,2987007.0,0,86529.0,999999.000,W,12695,490.000000,150.0,0.010802,226.000000,...,other,35.212434,other,other,other,other,other,other,other,other
8,2987008.0,0,86535.0,15.000,H,2803,100.000000,150.0,0.010802,226.000000,...,mobile safari 11.0,32.000000,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009.0,0,86536.0,117.000,W,17399,111.000000,150.0,0.023952,224.000000,...,other,35.212434,other,other,other,other,other,other,other,other
